## 예제 3-4) 사람의 손글씨 데이터인 MNIST를 이용해 MLP 설계할 때 Dropout & ReLU & Batch Normalization & He Uniform Initialization 적용해보기

일반적인 딥러닝 모델은 다음 순서대로 설계해 학습하고 성능을 평가함.  
1. 모델 구조를 설계하고 설계된 모델 구조의 파라미터 값을 랜덤으로 샘플링  
2. Feature 값으로 이용되는 데이터를 설계한 모델의 Input으로 사용해 Output을 계산  
3. 계산된 Output을 Input으로 이용한 Feature 값과 매칭되는 레이블 값을 기존에 정의한 objective function을 통해 Loss 값으로 계산  
4. 계산된 Loss 값을 통해 Gradient를 계산해 모델 내 파라미터 값을 Back Propagation에 의해 업데이트  
5. 이를 반복해 학습을 진행하며 완성된 모델의 성능 평가  

이 중 1번에서 설계한 모델 구조의 파라미터 값을 랜덤으로 샘플링하는 과정에서 어떤 분포에서 샘플링을 진행하는지에 따라 모델의 학습이 좋은 방향으로 갈수도, 나쁜 방향으로 진행될 수도 있음.  
-> 즉, 학습의 시작점을 좋게 설정하면 학습을 수월하게 할 수 있음!  

현재 파이토치 내의 nn.linear는 Output으로 계산되는 벡터의 차원 수의 역수 값에 대한 +/- 범위 내 Uniform Distribution을 설정해 샘플링함.  
이번 예제에서는, He Initialization으로 초기화해보자. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn                           
import torch.nn.functional as F
from torchvision import transforms, datasets 

In [2]:
if torch.cuda.is_available() :
    DEVICE = torch.device('cuda')
else :
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version : ', torch.__version__, ' Device : ', DEVICE)

Using PyTorch version :  1.9.0  Device :  cpu


/home/nyongja/anaconda3/envs/pytorch_study/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
train_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = True,
                              download = True,
                              transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = False,
                              transform = transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

/home/nyongja/anaconda3/envs/pytorch_study/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


nn.BatchNorm( ) 함수를 적용하는 부분은 논문 / 코드에 따라 Activation Function 전, 후가 달라질 수 있음.  
이 예제에서는 이전에 적용해보자.

In [5]:
class Net(nn.Module) : 
    def __init__(self) :  
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout_prob = 0.5
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)
        
    def forward(self, x) :
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [6]:
import torch.nn.init as init

Weight, Bias 등 딥러닝 모델에서 초깃값으로 설정되는 요소에 대한 모듈인 init을 임포트

In [7]:
def weight_init(m) :                              # (1)
    if isinstance(m, nn.Linear) :                 # (2)
        init.kaiming_uniform_(m.weight.data)      # (3)

(1) MLP 모델 내의 Weight를 초기화할 부분을 설정하기 위한 함수 정의  
(2) MLP 모델을 구성하고 있는 파라미터 중 nn.Linear에 해당하는 파라미터 값에 대해서만 지정  
(3) nn.Linear에 해당하는 파라미터 값에 대해 he_initialization을 이용해 파라미터 값을 초기화

In [8]:
model = Net().to(DEVICE)
model.apply(weight_init)                                                      # (1)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
criterion = nn.CrossEntropyLoss()

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


(1) 위에서 정의한 weight_init 함수를 Net( ) 클래스의 인스턴스인 model에 적용  

지금까지 다룬 예제가 Class 내 모델을 설계하는 영역에서 설정했다면 초기값은 모델 정의하는 부분에서 설정 해야 함.  
우선 model을 정의한 후 apply를 이용해 모델의 파라미터를 초기화.  
초기화 진행 시, 정의된 weight_init 함수를 보면 모델 내 파라미터 값 중 nn.Linear 인스턴스에 대해서는 kaiming_uniform_을 이용해 초기화하는 것으로 설정돼있음. (kaming_uniform_ = He Initialization)  
이외 파라미터 값은 기본값으로 설정된 분포에서 샘플링해 랜덤으로 설정됨.

In [9]:
def train(model, train_loader, optimizer, log_interval) :
    model.train()            
    for batch_idx, (image, label) in enumerate(train_loader) :
        image = image.to(DEVICE)                                                               
        label = label.to(DEVICE)                                                               
        optimizer.zero_grad()                                                                  
        output = model(image)                                                                  
        loss = criterion(output, label)                                                        
        loss.backward()                                                                        
        optimizer.step()                                                                       
        
        if batch_idx % log_interval == 0 :
            print("Train Eppoch : {} [{}/{}({:.0f}%)]\tTrain Loss : {:.6f}".format(
                Epoch, batch_idx * len(image), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def evaluate(model, test_loader) :
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad() :
        for image, label in test_loader :
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)  
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item() 
            
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy

In [11]:
for Epoch in range(1, EPOCHS + 1) :
    train(model, train_loader, optimizer ,log_interval = 200)    # (1)
    test_loss, test_accuracy = evaluate(model, test_loader)      # (2)
    print("\n[EPOCH : {}], \tTest Loss : {:.4f}, \tTest Accuracy : {:.2f} %\n".format(Epoch, test_loss, test_accuracy))

Train Eppoch : 1 [0/60000(0%)]	Train Loss : 3.002760
Train Eppoch : 1 [6400/60000(11%)]	Train Loss : 0.786273
Train Eppoch : 1 [12800/60000(21%)]	Train Loss : 0.494366
Train Eppoch : 1 [19200/60000(32%)]	Train Loss : 0.916311
Train Eppoch : 1 [25600/60000(43%)]	Train Loss : 0.384322
Train Eppoch : 1 [32000/60000(53%)]	Train Loss : 0.736573
Train Eppoch : 1 [38400/60000(64%)]	Train Loss : 0.673953
Train Eppoch : 1 [44800/60000(75%)]	Train Loss : 0.497907
Train Eppoch : 1 [51200/60000(85%)]	Train Loss : 0.479082
Train Eppoch : 1 [57600/60000(96%)]	Train Loss : 0.476468

[EPOCH : 1], 	Test Loss : 0.0069, 	Test Accuracy : 93.47 %

Train Eppoch : 2 [0/60000(0%)]	Train Loss : 0.423151
Train Eppoch : 2 [6400/60000(11%)]	Train Loss : 0.363234
Train Eppoch : 2 [12800/60000(21%)]	Train Loss : 0.235161
Train Eppoch : 2 [19200/60000(32%)]	Train Loss : 0.676439
Train Eppoch : 2 [25600/60000(43%)]	Train Loss : 0.159210
Train Eppoch : 2 [32000/60000(53%)]	Train Loss : 0.452457
Train Eppoch : 2 [38400